In [ ]:
# clip_retrieval_basic.py
import glob, torch, requests
from PIL import Image
import open_clip

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 1) Model
model, _, preprocess = open_clip.create_model_and_transforms("ViT-B-32", pretrained="openai")
tokenizer = open_clip.get_tokenizer("ViT-B-32")
model = model.to(DEVICE).eval()

# 2) Ảnh: dùng 3 ảnh từ web (bạn có thể thay thành thư mục local)
urls = [
    "https://ultralytics.com/images/bus.jpg",
    "https://ultralytics.com/images/zidane.jpg",
    "https://ultralytics.com/images/bicycle.jpg"
]
images = [preprocess(Image.open(requests.get(u, stream=True).raw).convert("RGB")) for u in urls]
images = torch.stack(images).to(DEVICE)   # [N,3,224,224]

# 3) Câu truy vấn (text queries)
queries = [
    "một chiếc xe buýt màu vàng",
    "một người đang chơi bóng đá",
    "một chiếc xe đạp trên đường phố"
]
text = tokenizer(queries).to(DEVICE)

# 4) Tính embedding & chuẩn hoá
with torch.no_grad():
    img_feat = model.encode_image(images);  img_feat = img_feat / img_feat.norm(dim=-1, keepdim=True)  # [N,D]
    txt_feat = model.encode_text(text);     txt_feat = txt_feat / txt_feat.norm(dim=-1, keepdim=True)  # [M,D]

# 5) Text→Image: tìm ảnh phù hợp với từng câu
scores = txt_feat @ img_feat.T   # [M, N]
print("\nText → Image (Top-1):")
for qi, q in enumerate(queries):
    top = int(scores[qi].argmax().item())
    print(f"- \"{q}\"  -->  best image: {urls[top]}")

# 6) Image→Text: tìm câu mô tả phù hợp với mỗi ảnh
scores2 = img_feat @ txt_feat.T  # [N, M]
print("\nImage → Text (Top-1):")
for ii, u in enumerate(urls):
    top = int(scores2[ii].argmax().item())
    print(f"- {u}  -->  best caption: \"{queries[top]}\"")
